In [13]:
# === SETUP & CONFIG (Using References folder) ===

from pathlib import Path
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import re, gzip, json, subprocess, sys

print("="*60)
print("OPTIMIZED MODE: P. cubensis References from Files")
print("="*60)
print("Reading references from: 03_Sequences_Paper/References/")
print("="*60 + "\n")

# --- Project roots ---
BASE = Path.cwd().resolve()
DATA_CANDIDATES = [
    BASE / "03_Sequences_Paper",
    BASE.parent / "03_Sequences_Paper",
]
for candidate in DATA_CANDIDATES:
    if candidate.exists():
        DATA = candidate
        break
else:
    raise FileNotFoundError("Unable to locate the 03_Sequences_Paper directory relative to this notebook.")

def resolve_dir(parent: Path, name: str) -> Path:
    """Resolve directory with optional whitespace handling"""
    direct = parent / name
    if direct.exists():
        return direct
    normalized = [child for child in parent.iterdir() if child.name.strip() == name]
    if normalized:
        return normalized[0]
    raise FileNotFoundError(f'Expected directory "{name}" under {parent}')

# --- Input folders ---
SCAFF_DIR = resolve_dir(DATA, "Assembly_scaffolds")
REF_SOURCE_DIR = resolve_dir(DATA, "References")  # Your reference FASTA files

# --- Output folders ---
REF_DIR = BASE / "reference_panels"
OUT_DIR = BASE / "cds_extraction_outputs"
REF_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

# --- Enzymes we care about ---
ENZYMES = ("PsiD", "PsiK", "PsiM", "PsiH")

# --- Build reference panels from your FASTA files ---
print("Building reference panels from FASTA files...")

REFS = {}
for enzyme in ENZYMES:
    # Create output directory for this enzyme
    enzyme_dir = REF_DIR / f"Proteins_{enzyme}"
    enzyme_dir.mkdir(parents=True, exist_ok=True)

    # Look for reference file for this enzyme
    ref_file = REF_SOURCE_DIR / f"{enzyme}_Psilocybe_cubensis.fasta"

    if not ref_file.exists():
        print(f"WARNING: Reference file not found: {ref_file}")
        continue

    # Read the reference FASTA file
    records = list(SeqIO.parse(ref_file, "fasta"))

    if not records:
        print(f"WARNING: No sequences found in {ref_file}")
        continue

    # Write to reference panel
    output_fasta = enzyme_dir / f"{enzyme}_refs.faa"
    SeqIO.write(records, output_fasta, "fasta")

    REFS[enzyme] = output_fasta
    print(f"  {enzyme}: {len(records)} sequence(s) -> {output_fasta}")

print(f"\nReference panels created: {len(REFS)}/{len(ENZYMES)} enzymes")
print("="*60 + "\n")

# Verify all enzymes have references
missing = set(ENZYMES) - set(REFS.keys())
if missing:
    raise FileNotFoundError(f"Missing references for: {', '.join(missing)}")

print("Setup complete! Ready to extract genes.")


OPTIMIZED MODE: P. cubensis References from Files
Reading references from: 03_Sequences_Paper/References/

Building reference panels from FASTA files...
  PsiD: 1 sequence(s) -> /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/reference_panels/Proteins_PsiD/PsiD_refs.faa
  PsiK: 1 sequence(s) -> /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/reference_panels/Proteins_PsiK/PsiK_refs.faa
  PsiM: 1 sequence(s) -> /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/reference_panels/Proteins_PsiM/PsiM_refs.faa
  PsiH: 1 sequence(s) -> /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/reference_panels/Proteins_PsiH/PsiH_refs.faa

Reference panels created: 4/4 enzymes

Setup complete! Ready to extract genes.


In [14]:
# === CELL 2: Single-Species Extraction (OPTIMIZED) ===

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import time

try:
    from tqdm.auto import tqdm
    TQDM_AVAILABLE = True
except ImportError:
    TQDM_AVAILABLE = False
    print("Warning: tqdm not installed. Progress bars disabled. Install with: pip install tqdm")

# --- OPTIMIZED PARAMETERS FOR SPEED ---
BESTN   = 1     # Best hit only (3x faster than bestn=3)
PERCENT = 30    # Minimal percent identity cutoff

# Optional soft length bands (AA) to prefer intact ORFs
AA_BANDS = {
    "PsiD": (350, 600),   # TDC typically ~480 aa
    "PsiK": (250, 420),   # kinase
    "PsiM": (180, 320),   # methyltransferase
    "PsiH": (400, 650),   # P450 ~510 aa
}

print(f"Extraction parameters: BESTN={BESTN}, PERCENT={PERCENT}")
print(f"Single reference mode + bestn=1 = ~3-4x faster!\n")

def run(cmd):
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if p.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(map(str,cmd))}\nSTDERR:\n{p.stderr}")
    return p.stdout

def index_fasta_dict(fasta_path):
    return {rec.id: str(rec.seq).upper().replace("-", "") for rec in SeqIO.parse(fasta_path, "fasta")}

def reverse_complement(seq):
    return str(Seq(seq).reverse_complement())

def translate_orf(seq):
    prot = str(Seq(seq).translate(to_stop=False))
    in_frame = (len(seq) % 3 == 0)
    has_internal_stop = "*" in prot[:-1]
    return prot, in_frame, has_internal_stop

def parse_exonerate_gff(gff_text):
    """
    Parse exonerate --showtargetgff output.
    Returns list of hits dicts with keys: seqid,strand,score,exons(list of (s,e,phase))
    """
    lines = [ln for ln in gff_text.splitlines() if ln and not ln.startswith("#")]
    hits, current = [], None
    for ln in lines:
        if "\t" not in ln: 
            continue
        seqid, source, feature, start, end, score, strand, phase, attrs = ln.split("\t")
        start, end = int(start), int(end)
        score = 0.0 if score == "." else float(score)
        f = feature.lower()
        if f == "gene":
            if current and current.get("exons"):
                hits.append(current)
            current = {"seqid": seqid, "strand": strand, "score": score, "exons": [], "attrs": attrs}
        elif f == "cds":
            if current is None:
                current = {"seqid": seqid, "strand": strand, "score": score, "exons": [], "attrs": attrs}
            ph = 0 if phase == "." else int(phase)
            current["exons"].append((start, end, ph))
    if current and current.get("exons"):
        hits.append(current)
    for h in hits:
        h["exons"].sort(key=lambda x: x[0])
    return hits

def stitch_cds(genome_dict, hit):
    seq_parts = [genome_dict[hit["seqid"]][s-1:e] for (s,e,ph) in hit["exons"]]
    cds = "".join(seq_parts)
    if hit["strand"] == "-":
        cds = reverse_complement(cds)
    # trim to multiple of 3 if needed
    if len(cds) % 3 != 0:
        cds = cds[:len(cds) - (len(cds) % 3)]
    return cds

def exonerate_protein2genome(ref_faa, scaff_fa, bestn=BESTN, minpct=PERCENT):
    return run([
        "exonerate", "--model", "protein2genome",
        "--showtargetgff", "yes", "--showalignment", "no", "--showvulgar", "no",
        "--bestn", str(bestn), "--percent", str(minpct),
        str(ref_faa), str(scaff_fa)
    ])

def score_candidate(enzyme, prot, in_frame, has_stop, score_raw, nt_len):
    """Lower is better; prefer in-frame, no stops, within AA band, higher score, longer."""
    aa_len = len(prot.rstrip("*"))
    band_pen = 0
    if AA_BANDS.get(enzyme):
        lo, hi = AA_BANDS[enzyme]
        if not (lo <= aa_len <= hi):
            band_pen = 1
    return (
        int(has_stop),          # 0 preferred
        int(not in_frame),      # 0 preferred
        band_pen,               # 0 preferred
        -score_raw,             # higher exonerate score preferred
        -nt_len,                # longer CDS preferred
    )

def run_species(scaff_fa_path, out_dir=OUT_DIR, pbar=None, verbose=True):
    """Extract PsiD/K/M/H for one species FASTA; returns a summary dict."""
    species = Path(scaff_fa_path).stem
    genome = index_fasta_dict(scaff_fa_path)
    species_out = out_dir / species
    species_out.mkdir(parents=True, exist_ok=True)
    summary = {"species": species, "results": []}

    # Count scaffolds for info
    n_scaffolds = len(genome)
    if verbose and pbar is None:
        print(f"[{species}] Processing genome with {n_scaffolds:,} scaffolds...")

    # Create enzyme-level progress bar if tqdm available
    enzyme_iter = REFS.items()
    if TQDM_AVAILABLE and pbar is None:  # only show enzyme bar in single-species mode
        enzyme_iter = tqdm(enzyme_iter, desc=f"{species}", leave=False, total=len(REFS))

    for enzyme, ref_path in enzyme_iter:
        if verbose and pbar is None:
            print(f"  [{species}] Processing {enzyme}...", end=" ", flush=True)
        
        start_time = time.time()
        candidates = []
        
        # Single reference mode - ref_path is a Path object, not a list
        if not ref_path.exists():
            if verbose and pbar is None:
                print(f"ERROR: Reference not found")
            summary["results"].append({
                "enzyme": enzyme,
                "status": "ERROR",
                "error": f"Reference file not found: {ref_path}"
            })
            continue
        
        gff_text = exonerate_protein2genome(ref_path, scaff_fa_path, bestn=BESTN, minpct=PERCENT)
        for h in parse_exonerate_gff(gff_text):
            cds = stitch_cds(genome, h)
            if not cds:
                continue
            prot, in_frame, has_stop = translate_orf(cds)
            cand = {
                "enzyme": enzyme, "score": h["score"], "nt_len": len(cds),
                "in_frame": in_frame, "has_stop": has_stop, "blocks": len(h["exons"]),
                "strand": h["strand"], "seqid": h["seqid"], "cds": cds, "prot": prot
            }
            candidates.append(cand)
        
        elapsed = time.time() - start_time
        if verbose and pbar is None:
            print(f"done ({elapsed:.1f}s)", flush=True)

        if not candidates:
            summary["results"].append({"enzyme": enzyme, "status": "NO_HIT"})
            continue

        # pick the best candidate (with bestn=1, usually only 1 candidate anyway)
        best = sorted(
            candidates,
            key=lambda c: score_candidate(enzyme, c["prot"], c["in_frame"], c["has_stop"], c["score"], c["nt_len"])
        )[0]

        # write outputs
        cds_rec  = SeqRecord(Seq(best["cds"]), id=f"{species}|{enzyme}", description=f"score={best['score']};blocks={best['blocks']};strand={best['strand']}")
        prot_rec = SeqRecord(Seq(best["prot"]), id=f"{species}|{enzyme}", description=f"in_frame={best['in_frame']};has_stop={best['has_stop']}")
        SeqIO.write([cds_rec],  species_out / f"{species}_{enzyme}.cds.fa",  "fasta")
        SeqIO.write([prot_rec], species_out / f"{species}_{enzyme}.prot.fa", "fasta")

        status = "OK" if (best["in_frame"] and not best["has_stop"]) else "CHECK_ORF"
        summary["results"].append({
            "enzyme": enzyme, "status": status, "score": best["score"],
            "nt_len": best["nt_len"], "blocks": best["blocks"], "strand": best["strand"], "seqid": best["seqid"]
        })

    # write per-species JSON summary
    with open((species_out / f"{species}_summary.json"), "w") as fh:
        json.dump(summary, fh, indent=2)
    
    # Update batch progress bar if provided
    if pbar is not None:
        pbar.update(1)
    else:
        print(f"[{species}] Done! Output → {species_out}")
    
    return summary

# === EXAMPLE: run one species ===
print("="*60)
print("TESTING: Single Species Extraction")
print("="*60 + "\n")

one_species = sorted(SCAFF_DIR.glob("Psilocybe_baeocystis*.fa*"))[0]
print(f"Test species: {one_species.name}\n")
test_summary = run_species(one_species, OUT_DIR)

# === VALIDATION: Compare with Example Outputs ===
print("\n" + "="*60)
print("VALIDATION: Comparing with Example Outputs")
print("="*60 + "\n")

EXAMPLE_DIR = BASE.parent / "02_ColabResults_Example"
if EXAMPLE_DIR.exists():
    species_name = one_species.stem
    our_out = OUT_DIR / species_name
    
    # Only PsiD has example output
    enzyme = "PsiD"
    
    # Compare CDS
    our_cds = our_out / f"{species_name}_{enzyme}.cds.fa"
    example_cds = EXAMPLE_DIR / "P_baeocystis_PsiD.cds.fa"
    
    # Compare Protein
    our_prot = our_out / f"{species_name}_{enzyme}.prot.fa"
    example_prot = EXAMPLE_DIR / "P_baeocystis_PsiD.prot.fa"
    
    if our_cds.exists() and example_cds.exists():
        our_cds_seq = str(list(SeqIO.parse(our_cds, "fasta"))[0].seq)
        example_cds_seq = str(list(SeqIO.parse(example_cds, "fasta"))[0].seq)
        
        print(f"📋 {enzyme} CDS Comparison:")
        print(f"   Our length:     {len(our_cds_seq):,} bp")
        print(f"   Example length: {len(example_cds_seq):,} bp")
        
        if our_cds_seq == example_cds_seq:
            print(f"   ✅ PERFECT MATCH - CDS sequences identical!\n")
        else:
            # Check if they're close
            from difflib import SequenceMatcher
            similarity = SequenceMatcher(None, our_cds_seq, example_cds_seq).ratio()
            print(f"   ⚠️  Sequences differ (similarity: {similarity*100:.1f}%)")
            
            # Show first difference
            for i, (a, b) in enumerate(zip(our_cds_seq, example_cds_seq)):
                if a != b:
                    print(f"   First difference at position {i}: {a} vs {b}\n")
                    break
    
    if our_prot.exists() and example_prot.exists():
        our_prot_seq = str(list(SeqIO.parse(our_prot, "fasta"))[0].seq)
        example_prot_seq = str(list(SeqIO.parse(example_prot, "fasta"))[0].seq)
        
        print(f"📋 {enzyme} Protein Comparison:")
        print(f"   Our length:     {len(our_prot_seq)} aa")
        print(f"   Example length: {len(example_prot_seq)} aa")
        
        if our_prot_seq == example_prot_seq:
            print(f"   ✅ PERFECT MATCH - Protein sequences identical!")
        else:
            similarity = SequenceMatcher(None, our_prot_seq, example_prot_seq).ratio()
            print(f"   ⚠️  Sequences differ (similarity: {similarity*100:.1f}%)")
            
            # Show first difference
            for i, (a, b) in enumerate(zip(our_prot_seq, example_prot_seq)):
                if a != b:
                    print(f"   First difference at position {i}: {a} vs {b}")
                    break
    
    print("\n" + "="*60)
    print("Validation complete!")
    print("="*60 + "\n")
else:
    print(f"⚠️  Example directory not found: {EXAMPLE_DIR}")
    print("   Skipping validation comparison.\n")


Extraction parameters: BESTN=1, PERCENT=30
Single reference mode + bestn=1 = ~3-4x faster!

TESTING: Single Species Extraction

Test species: Psilocybe_baeocystis_WTU-F-011245.scaffolds.fasta

[Psilocybe_baeocystis_WTU-F-011245.scaffolds] Processing genome with 111,256 scaffolds...


Psilocybe_baeocystis_WTU-F-011245.scaffolds:   0%|          | 0/4 [00:00<?, ?it/s]

  [Psilocybe_baeocystis_WTU-F-011245.scaffolds] Processing PsiD... done (4.0s)
  [Psilocybe_baeocystis_WTU-F-011245.scaffolds] Processing PsiK... done (3.8s)
  [Psilocybe_baeocystis_WTU-F-011245.scaffolds] Processing PsiM... done (3.7s)
  [Psilocybe_baeocystis_WTU-F-011245.scaffolds] Processing PsiH... done (5.4s)
[Psilocybe_baeocystis_WTU-F-011245.scaffolds] Done! Output → /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/cds_extraction_outputs/Psilocybe_baeocystis_WTU-F-011245.scaffolds

VALIDATION: Comparing with Example Outputs

📋 PsiD CDS Comparison:
   Our length:     618 bp
   Example length: 618 bp
   ✅ PERFECT MATCH - CDS sequences identical!

📋 PsiD Protein Comparison:
   Our length:     206 aa
   Example length: 206 aa
   ✅ PERFECT MATCH - Protein sequences identical!

Validation complete!



In [15]:
# === CELL 3: Batch Processing with Parallel Execution ===

import concurrent.futures as cf
import os
import csv
from datetime import datetime
from collections import Counter, defaultdict

all_scaff = sorted(SCAFF_DIR.glob("*.fa*"))
print(f"Total species to process: {len(all_scaff)}")

# ThreadPoolExecutor works in Jupyter (unlike ProcessPoolExecutor)
# Good for I/O-bound tasks like running exonerate subprocesses
max_workers = min(12, os.cpu_count() or 12)  # Adjust based on your system
print(f"Using {max_workers} parallel threads")
print("="*60 + "\n")

def process_one_species(scaff_path):
    """Worker function to process a single species"""
    try:
        return run_species(scaff_path, OUT_DIR, pbar=None, verbose=False)
    except Exception as e:
        import traceback
        return {
            "species": Path(scaff_path).stem,
            "error": str(e),
            "traceback": traceback.format_exc()
        }

started = datetime.now()
summaries = []

# Create progress bar
if TQDM_AVAILABLE:
    pbar = tqdm(total=len(all_scaff), desc="Processing species", unit="species")
else:
    pbar = None
    print(f"Processing {len(all_scaff)} species in parallel...")

# Parallel processing with ThreadPoolExecutor
with cf.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    future_to_species = {
        executor.submit(process_one_species, scaff): scaff.stem 
        for scaff in all_scaff
    }
    
    # Collect results as they complete
    for future in cf.as_completed(future_to_species):
        species_name = future_to_species[future]
        try:
            result = future.result()
            summaries.append(result)
            
            # Check for errors in result
            if "error" in result:
                print(f"\n✗ ERROR in {species_name}: {result['error']}")
            
            # Update progress
            if pbar:
                pbar.update(1)
            else:
                print(f"  [{len(summaries)}/{len(all_scaff)}] {species_name} - Done")
                
        except Exception as exc:
            print(f"\n✗ EXCEPTION in {species_name}: {exc}")
            summaries.append({
                "species": species_name,
                "error": str(exc)
            })
            if pbar:
                pbar.update(1)

if pbar:
    pbar.close()

elapsed = datetime.now() - started
print(f"\n{'='*60}")
print(f"Batch processing complete!")
print(f"Total time: {elapsed}")
print(f"Processed: {len(summaries)} species")
print(f"Average: {elapsed.total_seconds() / len(summaries):.1f} seconds per species")
print(f"{'='*60}\n")

# === Generate QC Reports ===
print("Generating QC reports...")

# Write extraction log
log_file = OUT_DIR / "extraction_log.json"
with open(log_file, "w") as fh:
    json.dump({
        "timestamp": started.isoformat(),
        "elapsed": str(elapsed),
        "total_species": len(all_scaff),
        "parallel_workers": max_workers,
        "summaries": summaries
    }, fh, indent=2)
print(f"  ✓ Extraction log: {log_file}")

# Collect problems for QC reports
problems = []
species_problem_counts = defaultdict(lambda: {"CHECK_ORF": 0, "NO_HIT": 0, "ERROR": 0})

for summary in summaries:
    species = summary.get("species", "unknown")
    
    # Handle species-level errors
    if "error" in summary:
        problems.append({
            "species": species,
            "enzyme": "ALL",
            "status": "ERROR",
            "details": summary["error"]
        })
        species_problem_counts[species]["ERROR"] += 1
        continue
    
    # Check each enzyme result
    for result in summary.get("results", []):
        enzyme = result.get("enzyme", "unknown")
        status = result.get("status", "UNKNOWN")
        
        if status in ["CHECK_ORF", "NO_HIT", "ERROR"]:
            details = result.get("error", f"Status: {status}")
            problems.append({
                "species": species,
                "enzyme": enzyme,
                "status": status,
                "details": details
            })
            species_problem_counts[species][status] += 1

# Write problems report
if problems:
    problems_tsv = OUT_DIR / "problems_report.tsv"
    with open(problems_tsv, "w", newline="") as fh:
        writer = csv.DictWriter(fh, fieldnames=["species", "enzyme", "status", "details"], delimiter="\t")
        writer.writeheader()
        writer.writerows(problems)
    print(f"  ✓ Problems report: {problems_tsv} ({len(problems)} issues)")
else:
    print("  ✓ No problems found - all extractions OK!")

# Write species problem summary
if species_problem_counts:
    summary_tsv = OUT_DIR / "species_problem_summary.tsv"
    with open(summary_tsv, "w", newline="") as fh:
        writer = csv.writer(fh, delimiter="\t")
        writer.writerow(["species", "total_problems", "CHECK_ORF", "NO_HIT", "ERROR"])
        for species in sorted(species_problem_counts.keys()):
            counts = species_problem_counts[species]
            total = sum(counts.values())
            writer.writerow([species, total, counts["CHECK_ORF"], counts["NO_HIT"], counts["ERROR"]])
    print(f"  ✓ Species summary: {summary_tsv} ({len(species_problem_counts)} species with issues)")

print(f"\n{'='*60}")
print("QC Reports Complete!")
print(f"{'='*60}")


Total species to process: 71
Using 12 parallel threads



Processing species:   0%|          | 0/71 [00:00<?, ?species/s]

Psilocybe_angustipleurocystidiata_ISOTYPE_NY-761597.scaffolds:   0%|          | 0/4 [00:00<?, ?it/s]

Psilocybe_argentina_SFSU-F-029894.scaffolds:   0%|          | 0/4 [00:00<?, ?it/s]

Psilocybe_azurescens_WTU-F19095-HOLOTYPE.scaffolds:   0%|          | 0/4 [00:00<?, ?it/s]

...

[Psilocybe_weilii_ISOTYPE_WTU-F-063525.scaffolds] Done! Output → /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/cds_extraction_outputs/Psilocybe_weilii_ISOTYPE_WTU-F-063525.scaffolds


Psilocybe_zapotecorum_FFCL689.scaffolds:   0%|          | 0/4 [00:00<?, ?it/s]

[Psilocybe_subpsilocybioides_HOLOTYPE_CFMR-PR-5689.scaffolds] Done! Output → /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/cds_extraction_outputs/Psilocybe_subpsilocybioides_HOLOTYPE_CFMR-PR-5689.scaffolds
[Psilocybe_subviscida_VPI-F-0003697.scaffolds] Done! Output → /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/cds_extraction_outputs/Psilocybe_subviscida_VPI-F-0003697.scaffolds


Psilocybe_zapotecorum_GAM00011076.scaffolds:   0%|          | 0/4 [00:00<?, ?it/s]

[Psilocybe_yungensis_SFSU-29944.scaffolds] Done! Output → /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/cds_extraction_outputs/Psilocybe_yungensis_SFSU-29944.scaffolds
[Psilocybe_quebecensis_NY-1901130.scaffolds] Done! Output → /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/cds_extraction_outputs/Psilocybe_quebecensis_NY-1901130.scaffolds
[Psilocybe_zapotecorum_FFCL689.scaffolds] Done! Output → /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/cds_extraction_outputs/Psilocybe_zapotecorum_FFCL689.scaffolds
[Psilocybe_xalapensis_ISOTYPE_NY-761630.scaffolds] Done! Output → /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/cds_extraction_outputs/Psilocybe_xalapensis_ISOTYPE_NY-761630.scaffolds
[Psilocybe_zapotecorum_GAM00011076.scaffolds] Done! Output → /Users/felix/Documents/20251015_Meeting_PsiloProteins_Felix/05_codex/cds_extraction_outputs/Psilocybe_zapotecorum_GAM00011076.scaffolds
[Psilocybe_caerulesce